In [70]:
# !pip install dataset
import torch
import torch.nn as nn
import torchvision
import einops
import pandas as pd
# import datasets
from datasets import Dataset, load_dataset, DatasetDict

class Attn(nn.Module):
    def __init__(self, n_heads, d_embed, d_hidden):
        super().__init__()

        self.output_lin = nn.Linear(in_features=d_hidden, out_features=d_embed)

        Q = nn.Linear(in_features=d_embed, out_features=d_hidden)
        nn.init.xavier_uniform()

# a = Attn()

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [71]:
# N, L, D, C = 1, 5, 3, 1
# N, C, L, D = 1, 1, 5, 3
N, L, D = 1, 5, 3
# sample_batched_input = torch.rand((N, C, L, D))
sample_batched_input = torch.rand((N, L, D))

sample_batched_input, sample_batched_input.shape#.unsqueeze(-1)

(tensor([[[0.2481, 0.6660, 0.8065],
          [0.0584, 0.1824, 0.4292],
          [0.7928, 0.0533, 0.6161],
          [0.9786, 0.0642, 0.2615],
          [0.7678, 0.6759, 0.9963]]]),
 torch.Size([1, 5, 3]))

In [72]:
nn.Linear(in_features=3, out_features=64)(sample_batched_input).shape

torch.Size([1, 5, 64])

In [73]:
# torch.nn.BatchNorm2d(affine=False, num_features=1)(
#     nn.Linear(in_features=3, out_features=64)(sample_batched_input)
#     ).shape

In [74]:
torch.nn.BatchNorm2d(affine=False, num_features=1)(torch.ones((2, 1, 2, 3)))
torch.nn.BatchNorm2d(affine=False, num_features=1)(torch.ones((2, 1, 2, 3)))

tensor([[[[0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.]]]])

In [75]:
torchvision.ops.MLP(in_channels=3, 
                    hidden_channels=[64, 128, 1024], 
                    norm_layer=lambda x : nn.BatchNorm1d(num_features=3),#torch.nn.GroupNorm(),#(num_features=3, affine=False), 
                    activation_layer=nn.ReLU)#.forward(input=sample_batched_input).shape

MLP(
  (0): Linear(in_features=3, out_features=64, bias=True)
  (1): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.0, inplace=False)
  (4): Linear(in_features=64, out_features=128, bias=True)
  (5): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Dropout(p=0.0, inplace=False)
  (8): Linear(in_features=128, out_features=1024, bias=True)
  (9): Dropout(p=0.0, inplace=False)
)

In [76]:
class MLP(nn.Module):
    def __init__(self, in_features, hidden_features, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        features = [in_features] + hidden_features

        # self.mlp = nn.ModuleList([
        #     # Layer
        #     nn.Sequential(
        #         nn.Linear(in_features=features[i], 
        #                   out_features=features[i+1]),
        #         nn.BatchNorm1d(num_features=3, affine=False),
        #         nn.ReLU(),
        #         nn.Dropout(p=0),
        #     )
        # self.mlp = nn.Sequential(nn.ModuleList([
        #     # Layer
        #     nn.Sequential(
        #         nn.Linear(in_features=features[i], 
        #                   out_features=features[i+1]),
        #         nn.BatchNorm1d(num_features=features[i], affine=False),
        #         nn.ReLU(),
        #         nn.Dropout(p=0),
        #     )
        # for i in range(0, len(features)-1)]))

        self.mlp_layers = nn.ModuleList([
            # Layer
            nn.Sequential(
                nn.Linear(in_features=features[i], 
                          out_features=features[i+1]),
                # nn.BatchNorm1d(num_features=features[i], affine=False),
                nn.ReLU(),
                nn.Dropout(p=0),
            )
        for i in range(0, len(features)-1)])
        
        # self.lin_1 = nn.Linear(in_features=in_features, out_features=hidden_features[0])
        # self.lin_2 = nn.Linear
    def forward(self, x):
        for layer in self.mlp_layers:
            x = layer(x)
        return self.mlp_layers(x)

# m = MLP(in_features=3, hidden_features=[64, 128, 1024])
# m(sample_batched_input)


In [77]:
torchvision.ops.MLP(in_channels=3, 
                    hidden_channels=[64, 128, 1024], 
                    norm_layer=nn.BatchNorm2d,#nn.RMSNorm,#lambda x : nn.BatchNorm1d(num_features=3),#torch.nn.GroupNorm(),#(num_features=3, affine=False), 
                    activation_layer=nn.ReLU)#(sample_batched_input)

MLP(
  (0): Linear(in_features=3, out_features=64, bias=True)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.0, inplace=False)
  (4): Linear(in_features=64, out_features=128, bias=True)
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Dropout(p=0.0, inplace=False)
  (8): Linear(in_features=128, out_features=1024, bias=True)
  (9): Dropout(p=0.0, inplace=False)
)

In [78]:
# class View(nn.Module):
#     def __init__(self, shape):
#         self.shape = shape

#     def forward(self, x):
#         return x.view(*self.shape)

    
class MLP_2(nn.Module):
    class TransposeChannelSequennce(nn.Module):
        def __init__(self, to_transpose=(-1, -2)):
            super().__init__()
            self.to_transpose = to_transpose

        def forward(self, x:torch.Tensor):
            return x.transpose(-1, -2)#**self.to_transpose)#-1, -2)
    def __init__(self, in_features, hidden_features, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        features = [in_features] + hidden_features + [in_features]

        self.mlp_layers = nn.ModuleList([
            # Layer
            nn.Sequential(
                nn.Linear(in_features=features[i-1], 
                          out_features=features[i]),
                TransposeChannelSequennce(),
                nn.BatchNorm1d(num_features=features[i], affine=False),
                TransposeChannelSequennce(),
                nn.ReLU(),
                nn.Dropout(p=0),
            ) if i < len(features)-1 else 
            nn.Sequential(
                nn.Linear(in_features=features[i-1], 
                          out_features=features[i]),
            )
        for i in range(1, len(features))])
        
        # self.lin_1 = nn.Linear(in_features=in_features, out_features=hidden_features[0])
        # self.lin_2 = nn.Linear
    def forward(self, x):
        for layer in self.mlp_layers:
            x = layer(x)
        return x#self.mlp_layers(x)

# m = MLP(in_features=3, hidden_features=[64, 128, 1024])
# m(sample_batched_input)


In [79]:
MLP_2(in_features=D, hidden_features=[64, 128, 1024])(sample_batched_input)

NameError: name 'TransposeChannelSequennce' is not defined

In [131]:
# nn.init.xavier_uniform(torch.zeros((1, 2)))
# 
# TransposeChannelSequennce()(TransposeChannelSequennce()(sample_batched_input)) #- sample_batched_input

tensor([[[0.0104, 0.9589, 0.0847],
         [0.6557, 0.1664, 0.9013],
         [0.2181, 0.5006, 0.2660],
         [0.6849, 0.1917, 0.3040],
         [0.7638, 0.0653, 0.5368]]])

In [113]:
# class View(nn.Module):
#     def __init__(self, shape):
#         self.shape = shape

#     def forward(self, x):
#         return x.view(*self.shape)

    
class MLP_3(nn.Module):
    class TransposeChannelSequennce(nn.Module):
        def __init__(self, to_transpose=(-1, -2)):
            super().__init__()
            self.to_transpose = to_transpose

        def forward(self, x:torch.Tensor):
            # print("doing tranpsose ", x.shape)
            return x.transpose(-1, -2)#**self.to_transpose)#-1, -2)
        #reul_at_end
    def __init__(self, in_features, hidden_features, relu_at_end=True, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.relu_at_end = relu_at_end
        features = [in_features] + hidden_features# + [in_features]

        self.mlp_layers = nn.ModuleList([
            # Layer
            nn.Sequential(
                nn.Linear(in_features=features[i-1], 
                          out_features=features[i]),
                self.TransposeChannelSequennce(),
                nn.BatchNorm1d(num_features=features[i], affine=False),
                self.TransposeChannelSequennce(),
                nn.ReLU() if i < len(features)-1 or relu_at_end else nn.Identity(),
                nn.Dropout(p=0),
            ) 
        for i in range(1, len(features))])
        
    def forward(self, x):
        for layer in self.mlp_layers:
            x = layer(x)
        return x#self.mlp_layers(x)
# torch.Tensor.squeeze()

In [114]:
mlp_samp = MLP_3(in_features=3, hidden_features=[64, 128, 4])(sample_batched_input)
mlp_samp, torch.nn.AdaptiveMaxPool1d(output_size=1)(mlp_samp.transpose(-1, -2)).squeeze(dim=-1).shape, mlp_samp.shape

(tensor([[[0.0000, 0.5756, 0.7706, 1.0622],
          [0.0000, 0.0000, 0.0000, 1.0143],
          [0.9853, 0.0000, 0.8041, 0.0000],
          [1.2562, 0.0000, 0.0000, 0.0000],
          [0.0000, 1.5638, 0.6488, 0.2108]]], grad_fn=<ReluBackward0>),
 torch.Size([1, 4]),
 torch.Size([1, 5, 4]))

In [115]:
# mlp_samp = MLP_3(in_features=3, hidden_features=[64, 128, 4])(sample_batched_input)
# mlp_samp, torch.nn.AdaptiveMaxPool1d(output_size=1)(mlp_samp.transpose(-1, -2)), mlp_samp.shape


class T2Net(nn.Module):
    def __init__(self, dim=3, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.mlp = MLP_3(in_features=dim, hidden_features=[64, 128, 1024])
        self.affine_T = nn.Linear(in_features=dim, out_features=dim, bias=True)
        self.global_max_pool = nn.AdaptiveMaxPool1d(output_size=1)
        
        # self.global_max_pool = nn.AdaptiveMaxPool1d(output_size=1)

    
        # Initialize the matrix in layer as identity
        with torch.no_grad():
            identity_weights = torch.eye(self.affine_T.weight.shape[0], dtype=float)
            self.affine_T.weight.copy_(identity_weights)
            
            # self.affine_T.bias.copy_(identity_weights)
            
        print(identity_weights)
        print(self.affine_T.weight)
        print(self.affine_T.bias)
        # nn.Linear

    """
    inp.shape: [N x L x D] , where N is batch, L is # of points, D is point-cloud dimension (dim of each point)
    """
    def forward(self, inp):
        return self.affine_T(inp) # TODO stub

In [116]:

class TNet(nn.Module):
    def __init__(self, dim=3, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.D = dim
        self.mlp = MLP_3(in_features=dim, hidden_features=[64, 128, 1024])
        self.affine_T = nn.Linear(in_features=1024, out_features=dim**2, bias=True)
        self.global_max_pool = nn.AdaptiveMaxPool1d(output_size=1)

        
        # self.global_max_pool = nn.AdaptiveMaxPool1d(output_size=1)

    
        # Initialize the matrix in layer as identity
        with torch.no_grad():
            identity_bias = torch.eye(dim, dtype=float).flatten()
            self.affine_T.bias.copy_(identity_bias)
            self.affine_T.weight.copy_(torch.zeros_like(self.affine_T.weight))
            
            # self.affine_T.bias.copy_(identity_weights)
            
        print(identity_bias)
        print(self.affine_T.weight)
        print(self.affine_T.bias)
        # nn.Linear

    """
    inp.shape: [N x L x D] , where N is batch, L is # of points, D is point-cloud dimension (dim of each point)
    """
    def forward(self, inp):
        inp = self.mlp(inp)
        inp = self.global_max_pool(inp.transpose(-1, -2)).squeeze(dim=-1)
        inp = self.affine_T(inp)
        inp = inp.reshape((self.D, self.D))#(self.affine_T.in_features, self.affine_T.in_features))
        return inp
        # return self.affine_T(inp) # TODO stub
    
# class 
    
t = TNet(dim=3)
tnet_out_samp = t(sample_batched_input)#(torch.tensor([1, 2, 3, 4]).float())

class PointNet(nn.Module):
    def __init__(self, dim=3, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.T1 = TNet(dim=dim)#3)
        self.T2 = TNet(dim=64)

        self.mlp1 = MLP_3(in_features=dim, hidden_features=[64, 64])
        self.mlp2 = MLP_3(in_features=64, hidden_features=[64, 128, 1024])


        # self.MLP_1 = torch.

        self.global_max_pool = nn.AdaptiveMaxPool1d(output_size=1)

    def forward(self, x):#, get_intermediate_for_segmentation=False):
        # First transform
        t_affine_1 = self.T1(x)
        x = torch.matmul(x, t_affine_1)

        # First mlp layer
        x = self.mlp1(x)

        # Second transform
        t_affine_2 = self.T2(x)
        x = torch.matmul(x, t_affine_2)
        t_affine_2_transformed = x.clone()

        # Second mlp + max pooling
        x = self.mlp2(x)
        # mlp2_out = x.clone()
        # x = self.global_max_pool(x.transpose(-1, -2)).squeeze(dim=-1)
        x = self.global_max_pool(torch.einsum('...ij->...ji', x)).squeeze(dim=-1)

        return {'maxpooled_out':x, 'intermediate_affine_out':t_affine_2_transformed}#'intermediate_mlp':mlp2_out}

        return x


# nn.MLP(n_hidden=9)

def identity_matrix(dim):
    return torch.eye(dim)


# class GlobalMaxPoll(nn.Module):
#     def __init__(self, *args, **kwargs) -> None:
#         super().__init__(*args, **kwargs)
print("first output:\n", tnet_out_samp)

tensor([1., 0., 0., 0., 1., 0., 0., 0., 1.], dtype=torch.float64)
Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)
Parameter containing:
tensor([1., 0., 0., 0., 1., 0., 0., 0., 1.], requires_grad=True)
first output:
 tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], grad_fn=<ViewBackward0>)


In [117]:

class PointNetForSegmentation(nn.Module):
    def __init__(self, num_classes, dim=3, pointnet=None, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.point_net = pointnet if pointnet else PointNet(dim=dim)
        self.mlp1 = MLP_3(in_features=1088, hidden_features=[512, 256, 128])
        self.mlp2 = MLP_3(in_features=128, hidden_features=[num_classes], relu_at_end=False)

    def forward(self, x):
        point_net_out = self.point_net(x)
        # batch,n_points = x.shape[0], x.shape[1]
        n_points = x.shape[1]

        concatted = torch.concat(
            (point_net_out['intermediate_affine_out'], einops.repeat(point_net_out['maxpooled_out'], 'b h -> b n h', n=n_points)),#torch.tile(point_net_out['maxpooled_out'].unsqueeze(), (1, n_points, ))), #.tile((n_points, ))
            dim=-1
        )
        # print(concatted)

        concatted = self.mlp1(concatted)
        concatted = self.mlp2(concatted)

        return concatted



In [118]:
point_net = PointNet(dim=3)

tensor([1., 0., 0., 0., 1., 0., 0., 0., 1.], dtype=torch.float64)
Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)
Parameter containing:
tensor([1., 0., 0., 0., 1., 0., 0., 0., 1.], requires_grad=True)
tensor([1., 0., 0.,  ..., 0., 0., 1.], dtype=torch.float64)
Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)
Parameter containing:
tensor([1., 0., 0.,  ..., 0., 0., 1.], requires_grad=True)


In [119]:
point_net(sample_batched_input)['intermediate_affine_out']#.shape#intermediate_mlp'].shape

tensor([[[1.6041, 0.1600, 0.0000, 1.6762, 1.0783, 1.3375, 1.6867, 0.0000,
          0.0000, 1.9546, 0.0000, 0.0000, 1.1513, 0.8312, 0.0000, 0.0024,
          0.4423, 0.0000, 1.2445, 0.7504, 1.6052, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.8367, 0.0000, 0.0000, 0.0000, 0.4787,
          0.0000, 0.0000, 0.2276, 0.9936, 0.0000, 0.0000, 0.0000, 0.0000,
          0.2538, 1.5712, 0.3534, 0.9825, 0.0000, 0.0000, 0.0000, 0.0000,
          0.9495, 0.0000, 0.0000, 0.0000, 0.2337, 0.0822, 0.0000, 0.0000,
          1.2649, 1.4419, 0.0000, 0.0000, 0.0451, 0.0000, 0.0000, 0.0000],
         [0.2067, 0.0000, 1.2395, 0.4682, 1.1236, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 1.2128, 0.3529, 0.0000, 0.0000, 1.8950, 1.7297,
          0.0000, 0.2871, 0.6073, 0.0000, 0.3994, 1.6820, 0.7648, 0.0000,
          1.9272, 1.5931, 0.0000, 0.0000, 0.2716, 0.0000, 1.4324, 1.3473,
          0.0000, 0.0000, 0.0000, 0.0000, 0.5164, 0.0000, 0.0251, 0.0000,
          0.0000, 0.7343, 0.0000, 0.0

In [120]:
"""testing regularization term"""
torch.linalg.matrix_norm(
            torch.eye(n=t.affine_T.weight.shape[0])
            -
            torch.matmul(t.affine_T.weight, t.affine_T.weight.T)
            )


tensor(3., grad_fn=<LinalgVectorNormBackward0>)

In [121]:
pointnet_seg = PointNetForSegmentation(dim=3, num_classes=8)

tensor([1., 0., 0., 0., 1., 0., 0., 0., 1.], dtype=torch.float64)
Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)
Parameter containing:
tensor([1., 0., 0., 0., 1., 0., 0., 0., 1.], requires_grad=True)
tensor([1., 0., 0.,  ..., 0., 0., 1.], dtype=torch.float64)
Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)
Parameter containing:
tensor([1., 0., 0.,  ..., 0., 0., 1.], requires_grad=True)


In [122]:
point_net(sample_batched_input)['intermediate_affine_out'].shape, torch.tile(point_net(sample_batched_input)['maxpooled_out'], (1, 5)).shape

(torch.Size([1, 5, 64]), torch.Size([1, 5120]))

In [123]:
# !pip install einops
import einops
einops.repeat(point_net(sample_batched_input)['maxpooled_out'], 'b c  -> b k c', k=5)#.shape


tensor([[[0.7522, 1.4407, 1.4489,  ..., 0.7545, 1.3683, 1.0689],
         [0.7522, 1.4407, 1.4489,  ..., 0.7545, 1.3683, 1.0689],
         [0.7522, 1.4407, 1.4489,  ..., 0.7545, 1.3683, 1.0689],
         [0.7522, 1.4407, 1.4489,  ..., 0.7545, 1.3683, 1.0689],
         [0.7522, 1.4407, 1.4489,  ..., 0.7545, 1.3683, 1.0689]]],
       grad_fn=<ExpandBackward0>)

In [124]:
pointnet_segout_one =  pointnet_seg(sample_batched_input)#pointnet_seg(sample_batched_input).shape, pointnet_seg(sample_batched_input)
pointnet_segout_one, pointnet_segout_one.shape

(tensor([[[-0.0878,  1.1475,  0.8187,  0.4445,  0.4567,  0.6572,  0.5177,
           -0.5355],
          [-1.7178,  1.0610,  0.6203, -1.6777,  0.6437, -0.9521, -1.3151,
           -0.5272],
          [ 0.5832, -0.1142, -0.2716, -0.3361, -1.4137,  0.1327, -0.0087,
           -1.1918],
          [-0.0793, -0.6281, -1.8466,  1.3517,  1.2358, -1.2662, -0.7538,
            1.6136],
          [ 1.3017, -1.4661,  0.6791,  0.2176, -0.9224,  1.4284,  1.5599,
            0.6410]]], grad_fn=<TransposeBackward0>),
 torch.Size([1, 5, 8]))

In [125]:
# f = torch.eye(5).unsqueeze(0).expand((1, 5, 8))
pointnet_segout_one = pointnet_segout_one#.flatten(-1)
label_path = torch.ones_like(pointnet_segout_one)
# nn.CrossEntropyLoss()(pointnet_segout_one, torch.softmax(pointnet_segout_one, dim=-1))
# nn.CrossEntropyLoss()(pointnet_segout_one, nn.LogSoftmax(dim=-1)(pointnet_segout_one))
# nn.CrossEntropyLoss()(pointnet_segout_one, torch.randint(size=(1, 5), high=7))
nn.CrossEntropyLoss(reduction='mean')(pointnet_segout_one.transpose(-1,-2), torch.randint(size=(1, 5), high=7))



# torch.softmax(pointnet_segout_one, dim=-1)
# nn.LogSoftmax(dim=-1)(pointnet_segout_one)

tensor(2.4827, grad_fn=<NllLoss2DBackward0>)

In [126]:
"""dataset loading"""
# pd.read_csv()
# with open("/Users/ishanshastri/Desktop/3D/pointnet/bag_dataset/expert_verified/points_label/1b9ef45fefefa35ed13f430b2941481.seg","r") as f:
#     loaded_list = f.read()
# loaded_tensor = torch.tensor(eval(loaded_list))

from numpy import genfromtxt
genfromtxt("/Users/ishanshastri/Desktop/3D/pointnet/bag_dataset/expert_verified/points_label/1b9ef45fefefa35ed13f430b2941481.seg").shape,\
genfromtxt("/Users/ishanshastri/Desktop/3D/pointnet/bag_dataset/points/1b9ef45fefefa35ed13f430b2941481.pts").shape



((2703,), (2703, 3))

In [127]:
genfromtxt("/Users/ishanshastri/Desktop/3D/pointnet/bag_dataset/expert_verified/points_label/1b9ef45fefefa35ed13f430b2941481.seg") - 1

array([1., 1., 1., ..., 1., 1., 0.])

In [128]:
import numpy as np
points_path = r'/Users/ishanshastri/Desktop/3D/pointnet/bag_dataset/points'
labels_path = r'/Users/ishanshastri/Desktop/3D/pointnet/bag_dataset/expert_verified/points_label'
import os

arr_x, arr_y = [], []

for fname in sorted(os.listdir(points_path)):
    x_path = os.path.join(points_path, fname)
    arr_x.append(genfromtxt(x_path))

for fname in sorted(os.listdir(labels_path)):
    label_path = os.path.join(labels_path, fname)
    arr_y.append(genfromtxt(label_path)-1)

# arr_x, arr_y = np.array(arr_x), np.array(arr_y)
# arr_x = np.array(arr_x)

In [129]:
import re
print(re.match("[*.seg]", os.listdir(labels_path)[0]))

None


In [130]:
# sorted(os.listdir(labels_path))
# {os.listdir(labels_path)[i]:i for i in os.listdir(labels_path)}
import re
# {re.match('(*.)[.seg]', i).groups(0):i for i in os.listdir(labels_path)}
# {re.match('(?P<name>*.)seg', i, re.DOTALL).groups(0):i for i in os.listdir(labels_path)}

re.match('(?P<name>.*)\.seg', "sdsfdsfdfd.seg")['name']
root_dict = {re.match('(?P<name>.*)\.seg', filename)['name'] : filename for filename in os.listdir(labels_path)}
# pt_dict = {re.match('(?P<name>.*)\.seg', filename)['name'] : filename for filename in os.listdir(points_path)}

# for k in root_dict:
#     root_dict[k] = 
# {re.match('(?P<name>.*)\.pts', filename)['name'] : filename for filename in os.listdir(labels_path)}
# for key in root_dict:
#     root_dict[key] = 
# root_dict
re.match('.*/(?P<name>.*)\.seg', "sdfdsf/f/f/fsdsfdsfdfd.seg")['name']


'fsdsfdsfdfd'

In [131]:
# np.stack(arr_x, axis=0).shape

In [132]:
arr_x[0].shape, arr_y[0].shape

((2712, 3), (2712,))

In [133]:
arr_x, arr_y = [], []
for name in os.listdir(labels_path):
    label_path = os.path.join(labels_path, name)
    pt_path = os.path.join(points_path, re.match('.*/(?P<name>.*)\.seg', label_path)['name']) + '.pts'
    # pt_path = os.path.join(points_path, re.sub('seg', 'pts', label_path))
    arr_y.append(genfromtxt(label_path)-1)
    arr_x.append(genfromtxt(pt_path))

In [134]:
arr_x[5].shape, arr_y[5].shape

((2822, 3), (2822,))

In [135]:
# torchvision.datasets.DatasetFolder(root=r'/Users/ishanshastri/Desktop/3D/pointnet/bag_dataset', loader=genfromtxt)

class PointDataset(torch.utils.data.Dataset):
    def __init__(self, arr_x, arr_y):
        self.arr_x = arr_x
        self.arr_y = arr_y

    def __len__(self):
        return len(self.arr_y)
    
    def __getitem__(self, idx):
        return {
            'x': torch.from_numpy((self.arr_x[idx]).astype(float)),
            'y': torch.from_numpy((self.arr_y[idx]).astype(float))
        }


In [136]:
my_point_dataset = PointDataset(arr_x=arr_x, arr_y=arr_y)

In [137]:
my_point_dataset[0]

{'x': tensor([[-0.0587,  0.0716,  0.1490],
         [ 0.0203, -0.2132,  0.3233],
         [ 0.0255, -0.3164,  0.3224],
         ...,
         [ 0.0388, -0.2028,  0.3196],
         [ 0.0418, -0.2234,  0.3188],
         [ 0.1038, -0.3267,  0.2927]], dtype=torch.float64),
 'y': tensor([0., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64)}

In [138]:
train_size = int(0.8*len(my_point_dataset))#point_dataloader_test
train_set, val_set = torch.utils.data.random_split(my_point_dataset, [train_size, len(my_point_dataset)-train_size])

point_dataloader_train = torch.utils.data.DataLoader(my_point_dataset, batch_size=1)#32)#torch.utils.data.DataLoader(my_point_dataset[0:int(train_size*len(my_point_dataset))])
point_dataloader_val = torch.utils.data.DataLoader(val_set, batch_size=1)#32)#torch.utils.data.DataLoader(my_point_dataset[int(train_size*len(my_point_dataset)):])

In [139]:
my_point_dataset[0]['y'].shape, my_point_dataset[0]['x'].shape

(torch.Size([2661]), torch.Size([2661, 3]))

In [160]:
reg_weight = 0.001
# def reg_loss(weights):
    
    # return nn.MSELoss(identity_matrix(dim=weights.shape[0]))

def train_epoch(model:PointNetForSegmentation, dataset):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

    # optimizer = torch.optim.adam()
    losses_total = []#0
    i=0
    for pt in dataset:
        # print(i)
        i+=1
        x, y = pt['x'].float(), pt['y'].float()
        optimizer.zero_grad()
        y_pred = model(x) # y_pred: (N, L, C) ; y: (N, L) (contains labels as class indices)

        # Regularization term
        # reg_loss = torch.linalg.matrix_norm(
        #     torch.eye(n=model.point_net.T2.affine_T.weight.shape[0])
        #     -
        #     torch.matmul(model.point_net.T2.affine_T.weight, model.point_net.T2.affine_T.weight.T)
        #     )

        # Loss computation -- we transpose y_pred to compute multi-K cross entropy which expects (N, C, L) 
        total_loss = loss_fn(y_pred.transpose(-1, -2), y.long()) #s+ reg_weight*reg_loss

        total_loss.backward()
        optimizer.step()

        losses_total.append(total_loss)

    return torch.tensor(losses_total).mean(dim=0), None#torch.mean(torch.tensor(losses_total))#losses_total/len(list(dataset)), 
        

        



In [161]:
model = PointNetForSegmentation(num_classes=2, dim=3)

tensor([1., 0., 0., 0., 1., 0., 0., 0., 1.], dtype=torch.float64)
Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)
Parameter containing:
tensor([1., 0., 0., 0., 1., 0., 0., 0., 1.], requires_grad=True)
tensor([1., 0., 0.,  ..., 0., 0., 1.], dtype=torch.float64)
Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)
Parameter containing:
tensor([1., 0., 0.,  ..., 0., 0., 1.], requires_grad=True)


In [162]:
num_epochs = 2
for i in range(num_epochs):
    loss, accuracy = train_epoch(model=model, dataset=point_dataloader_train)
    print(f"epoch {i}, loss: {loss}, accuracy: {accuracy}")

epoch 0, loss: 0, accuracy: tensor([0.7752, 0.9403, 0.6895, 0.6977, 0.7250, 0.7040, 0.6432, 0.6063, 0.5801,
        0.7117, 0.5959, 0.7226, 0.5673, 0.5133, 0.5433, 0.5395, 0.5863, 0.5360,
        0.4916, 0.7564, 0.5232, 0.4757, 0.6982, 0.8347, 0.7339, 0.5600, 0.6434,
        0.7549, 0.6757, 0.6234, 0.5903, 0.5740, 0.6320, 0.7471, 0.5144, 0.6096,
        0.5206, 0.6549, 0.5678, 0.4814, 0.6306, 0.5369, 0.4584, 0.4693, 0.4081,
        0.8689, 0.8712, 0.5816, 0.4801, 0.4709, 0.6203, 0.6145, 0.8649, 0.8352,
        0.6370, 0.4491, 0.7993, 0.5462, 0.6165, 0.7093, 0.7198, 0.6946, 0.6387,
        0.5398, 0.6448, 0.5620, 0.5663, 0.5356, 0.6786, 0.6699, 0.4925, 0.6278,
        0.6160, 0.5491, 0.5645, 0.4913])
epoch 1, loss: 0, accuracy: tensor([0.4862, 0.9370, 0.4874, 0.5175, 0.6780, 0.6686, 0.5206, 0.5069, 0.5055,
        0.7329, 0.5142, 0.7216, 0.5226, 0.4987, 0.4611, 0.4833, 0.5187, 0.4764,
        0.4413, 0.7957, 0.4779, 0.4223, 0.7255, 1.0271, 0.7561, 0.4769, 0.5900,
        0.9270, 0.6766,

In [169]:
from torch.profiler import profiler, record_function, ProfilerActivity, profile
with profile(activities=[ProfilerActivity.CPU], record_shapes=True, with_stack=True) as prof:
    with record_function("train_step"):
        model(sample_batched_input)
        print(prof.key_averages())
# model(sample_batched_input)#, sample_batched_input.shape

RuntimeError: Profiler didn't finish running

In [163]:
list(range(1,16))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]